In [ ]:
import os
import glob
import pandas as pd

In [ ]:
path = '../data/input/5_TFBS_score_subset_30May2018'
all_csvs = glob.glob(path + '/*.csv')
frame_list = []
for csv_ in all_csvs:
    test_frame = pd.read_csv(csv_, sep='\t')
    frame_list.append(test_frame)
    test_frame = pd.concat(frame_list)
# test_frame[test_frame['raw_position'] == 1]
#test_frame.head(n=20)

In [ ]:
test_frame.set_index('Unnamed: 0', drop=True, inplace=True)
test_frame.index.name = 'index'
# test_frame[test_frame['species'] == 'VT18492|0|MEMB002A|+|2283']

In [ ]:
test_frame = test_frame[test_frame['strand'] == 'positive']

In [ ]:
base_pairs = {'A': [1, 0, 0, 0], 
'C': [0, 1, 0, 0],
'G': [0, 0, 1, 0],
'T': [0, 0, 0, 1],
'a': [1, 0, 0, 0],
'c': [0, 1, 0, 0],
'g': [0, 0, 1, 0],
't': [0, 0, 0, 1],
'n': [0, 0, 0, 0],
'N': [0, 0, 0, 0]}

def align_sequence(length, sequence):
    if len(sequence) > length:
        aligned_seq = sequence[:length]
    else:
        aligned_seq = sequence + [0]*(length-len(sequence))
    return aligned_seq

def process_seq_record(seq_record, X, y, length_read):
    header = seq_record.description.split('|')
    expressed = int(header[1])
    y.append(expressed)
    # unflattened_seq = to_positive_strand(header[3], seq_record.seq)
    # NO NEED to reverse complement
    unflattened_seq = [base_pairs[n] for n in seq_record.seq]
    flattened_seq = [i for x in unflattened_seq for i in x]
    aligned_seq = align_sequence(length_read, flattened_seq)
    X.append(np.array(aligned_seq))

def prepare_input(training_size, test_size, length_read):
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    file_count = 0
    for file in os.listdir("../data/input/3.24_species_only"):
        if file.endswith(".fa"):
            if (file_count < training_size):
                read_file(file, X_train, y_train, length_read)
            elif (file_count < training_size + test_size):
                read_file(file, X_test, y_test, length_read)
            file_count += 1
    return X_train, y_train, X_test, y_test

def to_np_array(X_train, y_train, X_test, y_test):
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    if len(y_train.shape) == 1:
        y_train = np.transpose(np.array([y_train]))
    X_test = np.array(X_test)
    y_test = np.transpose(np.array(y_test))
    if len(y_test.shape) == 1:
        y_test = np.transpose(np.array([y_test]))
    return X_train, y_train, X_test, y_test

def read_file(file, X, y, length_read):
    seq_record_list = list(SeqIO.parse("../data/input/3.24_species_only/" + file,"fasta"))
    for i in range(len(seq_record_list)):
        process_seq_record(seq_record_list[i], X, y, length_read)

import os
from Bio import SeqIO

#Use the following dictionary to perform the transformation.
base_pairs = {'A': [1,0,0,0], 
'C': [0,1,0,0],
'G': [0,0,1,0],
'T': [0,0,0,1],
'a': [1,0,0,0],
'c': [0,1,0,0],
'g': [0,0,1,0],
't': [0,0,0,1],
'n': [0,0,0,0],
'N':[0,0,0,0]}
# one_hot = []
file_num_limit = 20    #The maximum number of files to be decoded
file_count = 0

# Not Used
def add_motif(lst, df):
    if df[df['motif_file'] == 'bcd_FlyReg.fm'].size != 0:
        lst.append(df[df['motif_file'] == 'bcd_FlyReg.fm']['score'].iloc[0])
    else:
        lst.append(0)
    if df[df['motif_file'] == 'hb_nar2008.fm'].size != 0:
        lst.append(df[df['motif_file'] == 'hb_nar2008.fm']['score'].iloc[0])
    else:
        lst.append(0)
    if df[df['motif_file'] == 'eve_SOLEXA.fm'].size != 0:
        lst.append(df[df['motif_file'] == 'eve_SOLEXA.fm']['score'].iloc[0])
    else:
        lst.append(0)

# Take the 'raw_position' and 'score' columns from the dataframe and create a dictionary
# mapping from 'raw_position to 'score'.
def score_dict(df, length):
    position_lst = df['raw_position'].tolist()
    score_lst = df['score'].tolist()
    motif_dict = dict(zip(position_lst, score_lst))
    for i in range(length):
        if i not in motif_dict:
            motif_dict[i] = 0
    return motif_dict
    
#Iterate through every file
for file in os.listdir("../data/input/3.24_species_only"):
    #When the number of file decoded has reached the limit, stop.
    one_hot = []
    if (file_count < file_num_limit):
        data = list(SeqIO.parse("../data/input/3.24_species_only/" + file,"fasta"))
        for n in range(0, len(data)):
            #Extract the header information
            header = data[n].description.split('|')
            descr = data[n].description
            regionID = header[0]
            expressed = header[1]
            speciesID = header[2]
            strand = header[3]
            #Complement all sequences in the negative DNA strand.
            if strand == '-':
                one_hot.append([descr, expressed, speciesID, [[e for e in base_pairs[n]] for n in data[n].seq.complement()]])
            else:
                one_hot.append([descr, expressed, speciesID, [[e for e in base_pairs[n]] for n in data[n].seq]])
        k = 1
        for item in one_hot:
            dna_len = len(item[3])
            test_frame2 = test_frame[test_frame['species'] == item[0]]
            frame_bcd = test_frame2
            frame_hb = test_frame2
            frame_eve = test_frame2
            dict_bcd = score_dict(frame_bcd, dna_len)
            dict_hb = score_dict(frame_hb, dna_len)
            dict_eve = score_dict(frame_eve, dna_len)
            i = 0
            for encoding in item[3]:
                encoding.append(dict_bcd[i])
                encoding.append(dict_hb[i])
                encoding.append(dict_eve[i])
                i += 1
            print(k)
            k += 1
        output = open("../data/output/" + regionID + ".txt", "w")
        output.write(str(one_hot))
        # one_hot.append(one_hot)
        output.close()
        file_count += 1
    else:
        break;

In [ ]:
def align_sequence(length, sequence):
    if len(sequence) > length:
        aligned_seq = sequence[:length]
    else:
        aligned_seq = sequence + [0]*(length-len(sequence))
    return aligned_seq

In [ ]:
def process_seq_record(seq_record, X, y, length_read):
    header = seq_record.description.split('|')
    expressed = int(header[1])
    y.append(expressed)
    # unflattened_seq = to_positive_strand(header[3], seq_record.seq)
    # NO NEED to reverse complement
    unflattened_seq = [base_pairs[n] for n in seq_record.seq]
    flattened_seq = [i for x in unflattened_seq for i in x]
    aligned_seq = align_sequence(length_read, flattened_seq)
    X.append(np.array(aligned_seq))

In [ ]:
import ast
def read_file(file, X, y, length_read):
    with open("../data/output" + file, "fasta") as f:
        seq_record_list = ast.literal_eval(f.read())
    for i in range(len(seq_record_list)):
        process_seq_record(seq_record_list[i], X, y, length_read)

In [ ]:
def prepare_input(training_size, test_size, length_read):
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    file_count = 0
    for file in os.listdir("../data/input/3.24_species_only"):
        if file.endswith(".fa"):
            if (file_count < training_size):
                read_file(file, X_train, y_train, length_read)
            elif (file_count < training_size + test_size):
                read_file(file, X_test, y_test, length_read)
            file_count += 1
    return X_train, y_train, X_test, y_test

def to_np_array(X_train, y_train, X_test, y_test):
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    if len(y_train.shape) == 1:
        y_train = np.transpose(np.array([y_train]))
    X_test = np.array(X_test)
    y_test = np.transpose(np.array(y_test))
    if len(y_test.shape) == 1:
        y_test = np.transpose(np.array([y_test]))
    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = prepare_input(200, 20, 4000)
X_train, y_train, X_test, y_test = to_np_array(X_train, y_train, X_test, y_test)
[X_train.shape, y_train.shape, X_test.shape, y_test.shape]

In [ ]:
from sklearn import linear_model as lm

In [ ]:
model = lm.LogisticRegression()
model.fit(X_train, y_train.ravel())

In [ ]:
y_predicted = np.array(model.predict(X_test))
round(sum(y_test.ravel() == y_predicted)/y_test.shape[0], 3)

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model, Sequential

def train_nn(X_train, y_train, pr):
    model = Sequential()
    model.add(Dense(units=1000, activation='relu', input_dim=4000))
    model.add(Dense(units=400, activation='relu'))
    model.add(Dense(units=40, activation='relu'))
    model.add(Dense(units=10, activation='elu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer = 'SGD',
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy'])
    model.fit(X_train, y_train, batch_size=100, epochs=5, verbose = pr)
    return model

In [ ]:
def test_accuracy(model, X_test, y_test):
    result = model.predict(X_test)
    correct = list(np.apply_along_axis(lambda x: 0 if x<0.5 else 1, 1, result))==y_test.ravel()
    return round(sum(correct)/y_test.shape[0], 3)

In [ ]:
test_accuracy(train_nn(X_train, y_train, 1), X_test, y_test)